In [2]:
from mygrad.nnet.layers import dense, RecurrentUnit 
from mygrad.nnet.activations import relu
from mygrad import Tensor


In [3]:
import numpy as np
import csv

In [4]:
def unzip(l):
    return zip(*l)

In [ ]:
#Gets data from bitfinex exchange
with open("./bitstampUSD.csv", mode='r') as q:
    data=[]
    File=csv.reader(q, delimiter=',')
    for line in File:
        data.append(line)

        
 


In [ ]:
print(data[-1])

In [ ]:
timePriceVolume=list(unzip(data))


In [ ]:
time=timePriceVolume[0]
price=timePriceVolume[1]
volume=timePriceVolume[2]

In [ ]:
time=np.array(time,dtype=int)
price=np.array(price,dtype=float)
volume=np.array(volume,dtype=float)

In [ ]:
def get_average_price(time,price,volume):
    tprice=np.reshape(price[3:],(2489638,5))
    tvol=np.reshape(volume[3:],(2489638,5))
    voltot=np.sum(tvol,1)
    tTime=time[3::5]
    weightedPrice=np.einsum('ij,ij->i', tprice, tvol)/voltot
    return (weightedPrice, tTime)
    
    

In [ ]:
aPrice,Time=get_average_price(time,price,volume)

In [ ]:
print(aPrice[:20])
print(price[:100])

timeData=Time[-250000]
aPriceData=aPrice[-250000:]
price=0
volume=0
time=0
data=0

In [ ]:
print(aPriceData.shape)

In [ ]:
x_train=aPriceData[:195000]
x_val=aPriceData[195000:200000]
x_test=aPriceData[200000:]


In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
print(x_train[:100])

In [5]:
def loss(x,y_true):
    return (((x-y_true)**2).sum())/(len(x))
def grad_descent(t_params,rate):
    for w in t_params:
        w.data-=(w.grad*rate)
def get_reg(t_params,reg):
    tot=0
    for W in t_params:
        tot=tot + reg * (W**2).sum()
    return tot

In [ ]:
#Reshaping Arrays of consecutive data into clumps
xtrain=np.copy(x_train)
# xtrain=np.reshape(xtrain,(1950,100))
# np.random.shuffle(xtrain)
# xtrain=np.transpose(xtrain)



In [ ]:
print(xtrain.shape)

In [6]:
def get_clump(xtrain,clumpSize,sentenceSize):
    clump=[]
    for i in range(clumpSize):
        r=np.random.randint(0,len(xtrain)-sentenceSize)
        clump.append(xtrain[r:r+sentenceSize])
    clump=np.array(clump)
    clump=np.transpose(clump)
    clump=clump[...,np.newaxis]
    return clump

In [ ]:
c=get_clump(xtrain,200,100)

In [ ]:
np.save("xtrain",x_train)
np.save("xval",x_val)
np.save("xtest",x_test)


In [7]:
x_train=np.load("xtrain.npy")
x_val=np.load("xval.npy")
x_test=np.load("xtest.npy")

In [8]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(195000,)
(5000,)
(50000,)


In [13]:
#initailizes hyperparameters
#where d determines the number of trainable parameters for 1st layer
D=100
#where c is the context (1)
C=1 
#where P is the size of the prediction(output)
P=1
#where q determines the number of trainable parameters for 2nd layer
Q=10000
#where O is output size(1)
O=1
#Learning rate
rate=1e-7
#Regulation strength
reg=1e5
#Back Propigation Level
bp_lim=5
#sentence size
S=100
#clump size
N=200

In [14]:
#intitailizing trainable parameters

#DxD, where d determines the number of trainable parameters for 1st layer
W_0=Tensor(0.001 * np.random.randn(*(D,D)))

#CxD where c is the context (1)
U_0=Tensor(0.001 * np.random.randn(*(C,D)))

#DxP where P is the size of the prediction(output)
V_0=Tensor(0.001 * np.random.randn(*(D,P)))

#QxQ, where q determines the number of trainable parameters for 2nd layer
# W_1=Tensor(0.001 * np.random.randn(*(Q,Q)))

# #PxQ
# U_1=Tensor(0.001 * np.random.randn(*(P,Q)))

# #QxO where O is output size(1)
# V_1=Tensor(0.001 * np.random.randn(*(Q,O)))

#,W_1,U_1,V_1
paramList=[W_0,U_0,V_0]

In [15]:
#initailizes reccurent networks
recurrent0=RecurrentUnit(U_0,W_0,bp_lim)
#recurrent1=RecurrentUnit(U_1,W_1,bp_lim)


In [17]:
Loss=[]
for i in range(400):
    clump=get_clump(x_train,N,S)
    HiddenDescriptor0=recurrent0(clump)
    layer1=dense(HiddenDescriptor0,V_0)
    L=(loss(layer1,np.transpose(clump))+get_reg(paramList,reg))
    print("Hoowoo")
    L.backward()
    print("WooHoo")
    grad_descent(paramList,rate)
    
    L.no_recursion_null_grad
    W_0.null_gradients()
    U_0.null_gradients()
    Loss.append(L.data)
    print(i)
    print(L.data)


Hoowoo
WooHoo
0
3804876391100816.0
Hoowoo
WooHoo
1
3654148431502632.0
Hoowoo
WooHoo
2
3509437098159654.5
Hoowoo
WooHoo
3
3370520213954178.0
Hoowoo


KeyboardInterrupt: 

In [ ]:
n=np.arange(10,dtype=float).reshape((2,5))
tot=np.sum(n,1)
t=-np.arange(10,dtype=float).reshape((2,5))
print(n)
print(tot)
print(t)
print(np.einsum('ij,ij->i', n, t))

In [ ]:
a=np.arange(64).reshape((16,4))
print(a)
a=np.transpose(a)
print(a)
a=np.transpose(a)
print(a)

In [ ]:
a=np.full((3,4,5),2)
b=np.full((5,10),1)
print(np.dot(a,b).shape)